In [1]:
import torch 
import numpy as np 
import os 

In [2]:
data_root = 'HateXplain/Data/'

os.listdir (data_root)

['classes_two.npy',
 'post_id_divisions.json',
 'README.md',
 'classes.npy',
 'dataset.json']

In [3]:
import json 

with open(data_root + 'dataset.json') as f:
    data = json.load(f)

In [4]:
# data

In [5]:
data_np = np.load(data_root + 'classes.npy')

In [6]:
data_np

array(['hatespeech', 'normal', 'offensive'], dtype='<U10')

In [7]:
from datasets import  load_dataset

# ds = load_dataset('json' , data_files= data_root + 'dataset.json')

ds = load_dataset('hatexplain')


Repo card metadata block was not found. Setting CardData to empty.


ValueError: Couldn't infer the same data file format for all splits. Got {NamedSplit('train'): ('json', {}), NamedSplit('test'): (None, {})}

In [ ]:
ds

In [9]:
from datasets import load_dataset, DatasetDict

# Define a function to preprocess the dataset
def preprocess(example):
    # Rename the 'post_tokens' feature to 'text'
    example['text'] = ' '.join(example['post_tokens'])
    example['label'] = max(example['annotators']['label'], key=example['annotators']['label'].count)
    del example['post_tokens']
    del example['rationales']
    return example

# Apply the preprocessing function to each split of the dataset
dataset = DatasetDict({
    split: ds[split].map(preprocess)
    for split in ds.keys()
})



Map:   0%|          | 0/1 [00:00<?, ? examples/s]

KeyError: 'post_tokens'

In [25]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'annotators', 'text', 'gt'],
        num_rows: 15383
    })
    validation: Dataset({
        features: ['id', 'annotators', 'text', 'gt'],
        num_rows: 1922
    })
    test: Dataset({
        features: ['id', 'annotators', 'text', 'gt'],
        num_rows: 1924
    })
})

In [26]:
dataset['train'][0]

{'id': '23107796_gab',
 'annotators': {'label': [0, 2, 2],
  'annotator_id': [203, 204, 233],
  'target': [['Hindu', 'Islam'],
   ['Hindu', 'Islam'],
   ['Hindu', 'Islam', 'Other']]},
 'text': 'u really think i would not have been raped by feral hindu or muslim back in india or bangladesh and a neo nazi would rape me as well just to see me cry',
 'gt': 2}

In [28]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [29]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [30]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/15383 [00:00<?, ? examples/s]

Map:   0%|          | 0/1922 [00:00<?, ? examples/s]

Map:   0%|          | 0/1924 [00:00<?, ? examples/s]

In [32]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [31]:
id2label = {0: "hate", 1: "normal", 2: "offense"}
label2id = {"hate": 0, "normal": 1, "offense": 2}

In [40]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
import evaluate

accuracy = evaluate.load("accuracy")

import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [41]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/raid/biplab/saikat/miniconda3/envs/nlp/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.027700,0.831541,0.627859
2,0.797300,0.777595,0.663202
3,0.745600,0.751966,0.673597
4,0.708200,0.738874,0.674116
5,0.676300,0.733787,0.679314
6,0.659600,0.726912,0.681393
7,0.639500,0.730191,0.685031
8,0.624600,0.731702,0.685551
9,0.610300,0.729421,0.680873
10,0.594100,0.737818,0.682952


TrainOutput(global_step=19240, training_loss=0.6199671219887208, metrics={'train_runtime': 610.2461, 'train_samples_per_second': 504.157, 'train_steps_per_second': 31.528, 'total_flos': 4769813329650972.0, 'train_loss': 0.6199671219887208, 'epoch': 20.0})